In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

import re
import string

In [3]:
dfake = pd.read_csv('./dataset/Fake.csv')
dtrue = pd.read_csv('./dataset/True.csv')

In [4]:
dfake['class'] = 0
dtrue['class'] = 1

# Identify the quantity of entries and its columns
dfake.shape, dtrue.shape

((23481, 5), (21417, 5))

In [5]:
# Clean dataset
dfake_manual_testing = dfake.tail(10)
dfake.drop(dfake.tail(10).index, inplace=True)

dtrue_manual_testing = dtrue.tail(10)
dtrue.drop(dtrue.tail(10).index, inplace=True)

dfake.shape, dtrue.shape


((23471, 5), (21407, 5))

In [6]:
dfake_manual_testing['class'] = 0
dtrue_manual_testing['class'] = 1

/var/folders/c7/tm52vwnx08l5_0g9cj0f22pw0000gn/T/ipykernel_2138/2506886640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfake_manual_testing['class'] = 0
/var/folders/c7/tm52vwnx08l5_0g9cj0f22pw0000gn/T/ipykernel_2138/2506886640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtrue_manual_testing['class'] = 1


In [7]:
# View first 10 entries
dfake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [8]:
# View first 10 entries
dtrue_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [9]:
# Merge true and fake news dataset
data_merge = pd.concat([dfake, dtrue], axis=0)
data_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [11]:
# Remove unnecessary columns
data = data_merge.drop(['title', 'subject', 'date'], axis=1)
data.isnull().sum()


text     0
class    0
dtype: int64

In [12]:
data.sample(frac=1)

,text,class
5690,WASHINGTON (Reuters) - The U.S. Treasury said ...,1
256,WASHINGTON (Reuters) - The U.S. House of Repre...,1
17374,Holy smokes! That s billion with a B! Where di...,0
11459,CNN commentator Van Jones stood by his praise ...,0
2551,The media is so DONE taking Donald Trump s con...,0
...,...,...
451,"While in the United States, Special Investigat...",0
13058,These images were all taken from a Facebook pa...,0
20028,"ERBIL/KIRKUK, Iraq (Reuters) - The United Stat...",1
1457,Farmworkers were pulled from fields on Friday ...,0


In [13]:
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)
data.columns

Index(['text', 'class'], dtype='object')

In [14]:
data.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [15]:
# Convert to lowercase and remove unnecessary characters. Gives more weight to generate accurate results
def wordformat(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) # removes special characters 
    text = re.sub('https?://\S+|www\.\S+', '', text) # removes links
    text = re.sub('<.*?>+', '', text) # removes html tags
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # removes punctuation
    text = re.sub('\n', '', text) # removes new lines
    text = re.sub('\w*\d\w*', '', text) # removes words containing numbers
    return text

data['text'] = data['text'].apply(wordformat)

In [16]:
x = data['text']
y = data['class']
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.25)

In [17]:
tf = TfidfVectorizer()

xv_tr = tf.fit_transform(x_tr)
xv_ts = tf.transform(x_ts)


In [18]:
LR = LogisticRegression()
LR.fit(xv_tr, y_tr)

# Check for accuracy
pred_lr = LR.predict(xv_ts)
print("Accuracy: "+str(LR.score(xv_ts, y_ts)))
print("Classification Report: \n"+str(classification_report(y_ts, pred_lr)))

Accuracy: 0.9852941176470589
Classification Report: 
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5910
           1       0.98      0.99      0.98      5310

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [19]:
DT = DecisionTreeClassifier()
DT.fit(xv_tr, y_tr)

# Check for accuracy
pred_dt = DT.predict(xv_ts)
print("Accuracy: "+str(DT.score(xv_ts, y_ts)))
print("Classification Report: \n"+str(classification_report(y_ts, pred_dt)))

Accuracy: 0.9955436720142602
Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5910
           1       0.99      1.00      1.00      5310

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [20]:
GB = GradientBoostingClassifier(random_state=0)
GB.fit(xv_tr, y_tr)

# Check for accuracy
pred_gb = GB.predict(xv_ts)
print("Accuracy: "+str(GB.score(xv_ts, y_ts)))
print("Classification Report: \n"+str(classification_report(y_ts, pred_gb)))

Accuracy: 0.9947415329768271
Classification Report: 
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5910
           1       0.99      1.00      0.99      5310

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [21]:
RF = RandomForestClassifier(random_state=0)
RF.fit(xv_tr, y_tr)

# Check for accuracy
pred_RF = RF.predict(xv_ts)
print("Accuracy: "+str(RF.score(xv_ts, y_ts)))
print("Classification Report: \n"+str(classification_report(y_ts, pred_RF)))

Accuracy: 0.9870766488413547
Classification Report: 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5910
           1       0.98      0.99      0.99      5310

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [22]:
def labels(n):
    if n == 0:
        return "Fake!"
    elif n == 1:
        return "Not Fake!"
    
def news_detector(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordformat)

    new_x_test = new_def_test["text"]
    new_xv_test = tf.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)

    return print("LR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(
        labels(pred_LR[0]),
        labels(pred_DT[0]),
        labels(pred_GB[0]),
        labels(pred_RF[0])))
    

In [40]:
# Enter text on input box. You can test using the content from the 'sample.txt' file
news = str(input())
news_detector(news)

LR Prediction: Not Fake! 
DT Prediction: Not Fake! 
GB Prediction: Not Fake! 
RF Prediction: Not Fake!
